### Исследование производительности работы с PostgreSQL

In [8]:
import time
import uuid
import asyncpg

from functools import wraps
from random import randint, choice
from typing import Any
from asyncpg.connection import Connection as AsyncpgConnection

In [9]:
POSTGRESQL_DSN = 'postgresql://postgres@localhost/moviesdb'
connection: AsyncpgConnection = await asyncpg.connect(POSTGRESQL_DSN)

In [10]:
def benchmark(operation: str):
    def decorator(func):
        @wraps(func)
        async def wrapper(*args, **kwargs):
            start_time = time.perf_counter()
            func_result = await func(*args, **kwargs)
            end_time = time.perf_counter()
            avg_time = end_time - start_time
            print(f"Время выполнения функции {operation} заняло {avg_time} cек.")
            return func_result
        return wrapper
    return decorator

In [11]:
await connection.execute('''
    CREATE TABLE IF NOT EXISTS movies(
        id UUID PRIMARY KEY,
        event_name VARCHAR(255) NOT NULL,
        movie_id UUID NOT NULL,
        user_id UUID NOT NULL,
        score INT
    )
''')

'CREATE TABLE'

In [12]:
def generate_like() -> dict:
    return {
        'id': uuid.uuid4(),
        'event_name': 'like',
        'user_id': uuid.uuid4(),
        'movie_id': uuid.uuid4(),
        'score': randint(0, 10),
	}


def generate_bookmark() -> dict:
    return {
        'id': uuid.uuid4(),
        'event_name': 'bookmark',
        'user_id': uuid.uuid4(),
        'movie_id': uuid.uuid4(),
        'score': None,
	}


async def clear_table(
    table_name: str
):
    await connection.execute(f'''
        DELETE FROM public.{table_name}
    ''')


async def insert_rows_in_movies_table(
    rows: list[dict]
) -> None:
    column_names = ', '.join(rows[0].keys())
    column_values = [tuple(row.values()) for row in rows]
    placeholders = ', '.join(
        ['$' + str(i+1) for i in range(len(rows[0].keys()))]
    )

    await connection.executemany(f'''
        INSERT INTO public.movies ({column_names})
        VALUES ({placeholders})
    ''', column_values)


async def insert_row_in_movies_table(
    row: dict
) -> None:
    column_names = ', '.join(row.keys())
    column_values = tuple(row.values())
    placeholders = ', '.join(
        ['$' + str(i+1) for i in range(len(row.keys()))]
    )

    await connection.execute(f'''
        INSERT INTO public.movies ({column_names})
        VALUES ({placeholders})
    ''', *column_values)


async def get_random_rows(
    event_name: str,
    events_count: int = 10
) -> list[asyncpg.Record]:
    return await connection.fetch(f'''
        SELECT * FROM public.movies
        WHERE event_name = $1
        ORDER BY random()
        LIMIT {events_count}
    ''', event_name)


async def generate_data_in_postgresql(
    generator: Any,
    rows_count: int,
    batch_size: int = 100
) -> None:
    rows = []
    for _ in range(rows_count):
        rows.append(generator())
        if len(rows) >= batch_size:
            await insert_rows_in_movies_table(rows)
            rows.clear()
    if rows:
        await insert_rows_in_movies_table(rows)

In [13]:
@benchmark(operation='Получение количества лайков для конкретного фильма')
async def get_count_of_likes_for_movie(
    movie_id: uuid.UUID
) -> asyncpg.Record:
    return await connection.fetch('''
        SELECT COUNT(*) FROM public.movies
        WHERE event_name='like' AND movie_id = $1
    ''', movie_id)


@benchmark(operation='Получение списка закладок для рандомного пользователя')
async def get_user_bookmarks(
    user_id: uuid.UUID,
) -> list[asyncpg.Record]:
    return await connection.fetch('''
        SELECT * FROM public.movies
        WHERE event_name = 'bookmark' AND user_id = $1
    ''', user_id)


@benchmark(operation='Добавление лайка пользователя к фильму')
async def add_like_to_movie(
    user_id: uuid.UUID,
    movie_id: uuid.UUID
) -> None:
    await insert_row_in_movies_table(
		{
            'id': uuid.uuid4(),
            'event_name': 'like',
            'user_id': user_id,
            'movie_id': movie_id,
            'score': randint(0, 10)
        }
    )


@benchmark(operation='Удаление лайка пользователя для фильма')
async def delete_like_from_movie(
    user_id: uuid.UUID,
    movie_id: uuid.UUID
) -> None:
    await connection.execute('''
        DELETE FROM public.movies
        WHERE event_name = 'like' AND user_id = $1 AND movie_id = $2
    ''', user_id, movie_id)

In [14]:
await clear_table('movies')
for _ in range(100):
    await generate_data_in_postgresql(generate_like, 25_000, 25_000)
    await generate_data_in_postgresql(generate_bookmark, 25_000, 25_000)

random_like_rows = await get_random_rows('like', 10_000)
random_bookmark_rows = await get_random_rows('bookmark', 10_000)

await get_count_of_likes_for_movie(choice(random_like_rows)['movie_id'])
await get_user_bookmarks(choice(random_bookmark_rows)['user_id'])

random_user_id, random_movie_id = (uuid.uuid4(), uuid.uuid4())
await add_like_to_movie(random_user_id, random_movie_id)
await delete_like_from_movie(choice(random_like_rows)['user_id'], choice(random_bookmark_rows)['movie_id'])

Время выполнения функции Получение количества лайков для конкретного фильма заняло 0.2144864330002747 cек.
Время выполнения функции Получение списка закладок для рандомного пользователя заняло 0.2084355590013729 cек.
Время выполнения функции Добавление лайка пользователя к фильму заняло 0.004707498999778181 cек.
Время выполнения функции Удаление лайка пользователя для фильма заняло 0.5311659250000957 cек.
